In [1]:
import numpy as np
import scipy.special
import numpy as np
import itertools

In [2]:
np.random.seed(101)

In [3]:
N = 100
x = np.zeros((N, 10))

mu = 0
sigma = 0.5
weights = sigma * np.random.randn(10) + mu

for j in range(0, 10):
    x[:, j] =  np.random.rand(100)

In [92]:
m_min = 5000
m_max = x.shape[1] * m_min + x.shape[0] * 10
n = x.shape[1]

In [6]:
def f(X):
    return np.dot(X, weights) + 10

We would like to see the effect of feature 3

In [80]:
phi = np.zeros(weights.shape[0])
feature_explained = 4
instance_explained_idx = 30
instance_explained = x[instance_explained_idx]

print(instance_explained, f(instance_explained))

[0.7015073  0.71085279 0.79315227 0.43061039 0.88334184 0.21324185
 0.82244529 0.44761512 0.05476228 0.10747345] 11.666164750702935


In [81]:
b_instance_explained = np.where(instance_explained > 0, 1, 0)

```python
for _ in range(total_samples):
    perm = np.random.permutation(10)
    feature_explained_pos = np.where(perm == feature_explained)[0][0]
    b1 = np.concatenate((instance_explained[:feature_explained_pos], w[feature_explained_pos :])).flatten()
    b2 = np.concatenate((instance_explained[:feature_explained_pos - 1], w[feature_explained_pos -1 :])).flatten()
    phi[feature_explained] += f(b2) - f(b1)
phi[feature_explained] = phi[feature_explained] / total_samples
```

In [93]:
def get_phi(feature_explained, explicand): 
    perm = np.random.permutation(n)
    w = x[np.random.randint(N)]
    feature_explained_pos = np.where(perm == feature_explained)[0][0]
    b1 = np.concatenate((explicand[perm[:feature_explained_pos]], w[perm[feature_explained_pos:]])).flatten()
    b2 = np.concatenate((explicand[perm[:feature_explained_pos - 1]], w[perm[feature_explained_pos - 1:]])).flatten()
    
    return f(b2) - f(b1)

In [94]:
phi = np.zeros((n, m_min))
for i in range(n):
    for j in range(m_min):
        phi[i, j] =  get_phi(i, instance_explained)

In [95]:
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def shapley_kernel(M,s):
    if s == 0 or s == M:
        return 10000
    return (M-1)/(scipy.special.binom(M,s)*s*(M-s))

In [96]:
def kernel_shap(f, x, reference, M):
    X = np.zeros((2**M,M+1))
    X[:,-1] = 1
    weights = np.zeros(2**M)
    V = np.zeros((2**M,M))
    for i in range(2**M):
        V[i,:] = reference

    ws = {}
    for i,s in enumerate(powerset(range(M))):
        s = list(s)
        V[i,s] = x[s]
        X[i,s] = 1
        ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
        weights[i] = shapley_kernel(M,len(s))
    y = f(V)
    tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))
    return np.dot(tmp, np.dot(np.dot(X.T, np.diag(weights)), y))

In [97]:
reference = np.zeros(n)
phi_kshap = kernel_shap(f, instance_explained, reference, n)
base_value = phi_kshap[-1]
shap_values_kshap = phi_kshap[:-1]

In [98]:
for i in range(n):
    print('Feature {}, SHAP: {}, KSHAP: {}, Weight: {}'.format(i, np.mean(phi[i]), shap_values_kshap[i], weights[i]))

Feature 0, SHAP: 0.022603058790727146, KSHAP: 0.949437455736188, Weight: 1.353424919699969
Feature 1, SHAP: 0.018577364391551277, KSHAP: 0.22325494395045098, Weight: 0.3140663543922298
Feature 2, SHAP: 0.04911035671633056, KSHAP: 0.3600790119562589, Weight: 0.45398472323827155
Feature 3, SHAP: -0.015650773673936743, KSHAP: 0.10847630101488903, Weight: 0.2519128769111968
Feature 4, SHAP: 0.051381399291992286, KSHAP: 0.2875798640270659, Weight: 0.3255589739716343
Feature 5, SHAP: -0.03769589200084727, KSHAP: -0.03404598466295283, Weight: -0.15965902229651663
Feature 6, SHAP: 0.044308110633860115, KSHAP: -0.34874845865851967, Weight: -0.42403849170181573
Feature 7, SHAP: -0.012064676783584975, KSHAP: 0.13561962688254425, Weight: 0.3029826747474668
Feature 8, SHAP: -0.06101241824670022, KSHAP: -0.05525974347256124, Weight: -1.009084122018696
Feature 9, SHAP: -0.05929550561052307, KSHAP: 0.03977173396560829, Weight: 0.3700610285280534


In [99]:
np.abs(np.mean(phi, axis=1) - weights).sum(), np.abs(shap_values_kshap, - weights).sum()

(4.855711764769606, 2.542273124327039)